## Scrap the net to get information regarding Class Action Lawsuits

### Load the necessary libraries

In [54]:
from selenium.webdriver import Chrome
import selenium
from selenium import webdriver as wd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import numpy as np

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

In [39]:
# Initiate the chrome drivers

chrome_options = wd.ChromeOptions()

driver = wd.Chrome(executable_path='C:/Users/chromedriver.exe',options=chrome_options)
driver_2 = wd.Chrome(executable_path='C:/Users/chromedriver.exe',options=chrome_options)

### Scraped 'https://www.classaction.org/database' for all the lawsuits

#### A snippet of all the information that needs to be extracted for a single lawsuit
![title](Law.png)

In [40]:
# link to be scraped
links = ['https://www.classaction.org/database']

In [27]:
# Function to loop the Hide Details i.e. DEFENDANT(s), LAW(s), and STATE(s) 

def get_class(prev):
    if prev == 'Defendant':
        return 'Law'
    elif prev == 'Law':
        return 'State'
    else:
        return 'Defendant' 

In [68]:
# Initiating the necessary variables for storing the data

current_class = 'Defendant'
info = {'Defendant' : [], 'Law': [], 'State':[]}

Date =[]
Review = []
Case = []
URL = []
Blog = []
blog_text = ''

In [77]:
# Run it for all the 40 pages

for i in range(39):
        date = driver.find_elements_by_xpath('//span[@class="dib"]')
        review = driver.find_elements_by_xpath("//p[@class='measure-wide lh-copy mb0']")
        href = driver.find_elements_by_xpath("//a[contains(@href, '.pdf') and contains(@class, 'dib link')]")
        header = driver.find_elements_by_xpath('//h3[@class="mt0 mb2"]')
        details = driver.find_elements_by_xpath('//p[@class="f6 mt2 mb0"]')
        blog_div = driver.find_elements_by_xpath('//div[@class="dtc-l w-100 w-25-l tc pv2 pv0-l tr-l"]')
        
        # Extract the date
        for i in date:
            Date.append(i.text)
        
        # Extract the reviews
        for i in review:
            Review.append(i.text)
        
        # Extract the header into Case
        for i in header:
            Case.append(i.text)
        
        # Extract the pdf urls
        for i in href:
            URL.append(i.get_attribute('href'))
        
        # Extract the hidden details
        for i in details:
            info[current_class].append(i.text)
            current_class = get_class(current_class)
        
        # Extract the blogs
        for i in blog_div:
            blog_a = i.find_elements_by_xpath('.//a[@class="dib br-pill ba b--near-white-blueish bg-near-whitest-blueish no-underline ph3 pv1 f6 midnight hover-green"]')
            try:
                text_link = blog_a[0].get_attribute('href')
                driver_2.get(text_link)
                text = driver_2.find_elements_by_xpath("//p")
                for t in text:
                    if t.get_attribute('class') == '':
                        #print(t.text)
                        blog_text= blog_text + t.text + ' '
            except:
                pass
             
            Blog.append(blog_text) 
            blog_text = ''      
        
        next_button = driver.find_element_by_xpath('//a[@class = "ais-Pagination-link" and @aria-label = "Next page"]')
        next_button.get_attribute('href')
        next_button.click()

In [70]:
# Total number of cases extracted

len(Case)

1000

In [78]:
# Store the data extracted in a data frame

df = pd.DataFrame({ 'Case':Case , 'Date':Date,'Review':Review,  
                   'State': info['State'], 'Law': info['Law'], 'PDF Link':URL,
                    'Article': Blog })


In [79]:
df.head(5)

,Case,Date,Review,State,Law,PDF Link,Article
0,"Grillo et al. v. RCN Telecom Services, LLC et al.",FILED: 07/10/2020,A class action claims RCN has pulled a bait-an...,New Jersey,New Jersey Consumer Fraud ActNew Jersey Truth-...,https://www.classaction.org/media/grillo-et-al...,A proposed class action alleges RCN and a numb...
1,Kraft et al. v. Essentia Health et al.,FILED: 07/10/2020,A class action claims Essentia Health and yet-...,North Dakota,,https://www.classaction.org/media/kraft-et-al-...,Two North Dakota parents allege Essentia Healt...
2,"Endeavor Trading, LLC v. JPMorgan Chase & Co. ...",FILED: 07/09/2020,A class action claims JPMorgan unlawfully mani...,New York,,https://www.classaction.org/media/endeavor-tra...,A proposed class action claims JPMorgan Chase ...
3,"Mendiola v. Home Depot U.S.A., Inc. et al.",FILED: 07/09/2020,Home Depot is alleged in a class action to hav...,Florida,ERISA,https://www.classaction.org/media/mendiola-v-h...,An ex-Home Depot employee claims COBRA notices...
4,"Hicks v. Evergreen Living & Rehab Center, LLC",FILED: 07/09/2020,A former Evergreen Living & Rehab Center emplo...,Illinois,Illinois Biometric Information Privacy Act,https://www.classaction.org/media/hicks-v-ever...,"Evergreen Living & Rehab Center, LLC faces a p..."


In [80]:
# Extract the Plaintiff and defendants from the case header

df[['Plaintiff', 'Defendant']] = df.Case.str.split(" v.", expand  =  True)

In [81]:
# Remove 'FILED:' from the Date

df['Date'] = df.Date.str.split(':').str[1]

In [75]:
# Store the data fram to a csv file

df.to_csv('Final_lawsuits_7_13_2020.csv',index=False)

In [82]:
df.head(5)

,Case,Date,Review,State,Law,PDF Link,Article,Plaintiff,Defendant
0,"Grillo et al. v. RCN Telecom Services, LLC et al.",07/10/2020,A class action claims RCN has pulled a bait-an...,New Jersey,New Jersey Consumer Fraud ActNew Jersey Truth-...,https://www.classaction.org/media/grillo-et-al...,A proposed class action alleges RCN and a numb...,Grillo et al.,"RCN Telecom Services, LLC et al."
1,Kraft et al. v. Essentia Health et al.,07/10/2020,A class action claims Essentia Health and yet-...,North Dakota,,https://www.classaction.org/media/kraft-et-al-...,Two North Dakota parents allege Essentia Healt...,Kraft et al.,Essentia Health et al.
2,"Endeavor Trading, LLC v. JPMorgan Chase & Co. ...",07/09/2020,A class action claims JPMorgan unlawfully mani...,New York,,https://www.classaction.org/media/endeavor-tra...,A proposed class action claims JPMorgan Chase ...,"Endeavor Trading, LLC",JPMorgan Chase & Co. et al.
3,"Mendiola v. Home Depot U.S.A., Inc. et al.",07/09/2020,Home Depot is alleged in a class action to hav...,Florida,ERISA,https://www.classaction.org/media/mendiola-v-h...,An ex-Home Depot employee claims COBRA notices...,Mendiola,"Home Depot U.S.A., Inc. et al."
4,"Hicks v. Evergreen Living & Rehab Center, LLC",07/09/2020,A former Evergreen Living & Rehab Center emplo...,Illinois,Illinois Biometric Information Privacy Act,https://www.classaction.org/media/hicks-v-ever...,"Evergreen Living & Rehab Center, LLC faces a p...",Hicks,"Evergreen Living & Rehab Center, LLC"
